# Jupyter Notebook UI to generate TAP data for MWT_Dashboards!

### Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

**Run all cells/steps sequentially, even the ones that do not need input**


## Step-by-Step Analysis of the Jupyter Notebook

| Step | Purpose | Key Actions |
|------|---------|-------------|
| **1. Import Packages** | Load required Python libraries for data analysis | Imports `pandas`, `numpy`, `matplotlib`, etc. | 
| **2. Pick Filepath** | User input: select folder containing `.trv` files | Uses `FileChooser` widget to select directory | 
| **3. User-Defined Variables** | Set experiment parameters | Defines: `number_of_taps`, `ISI`,`first_tap`; Calculates `tolerances` (time windows for taps) | 
| **4. Construct Filelist** | Find all `.trv` files in selected folder | Sets working directory and scans `folder_path` using; Displays no. of `.trv` files found in the folder |
| **5. Process Data Function** | Define functions to load and clean `.trv` data to use in step 6| - `ProcessData()`: Loads files, calculates metrics (reversal probability, speed)<br>- `assign_taps()`: Labels data with tap numbers <br>- `insert_plates()` |
| **6.1 Process Data (Input)** | Verify strain identifiers from file paths | Checks `filelist` for unique strain names (e.g., "N2") | `StrainNames` dictionary (maps strains to IDs) |
| **6.2 Process Data (Run)** | Apply processing to all strains | Runs `ProcessData()` and `assign_taps()` for each strain | `DataLists` with cleaned DataFrames per strain |
| **7. Grouping & Naming** | Combine data from all strains | - Concatenates DataFrames<br>- Assigns dataset names (e.g., "N2") | `TotalConcatenated` DataFrame with all processed data |
| **Output CSV** | Save processed data | Exports `TotalConcatenated` to CSV (e.g., `PD_Screen_tap_output.csv`) | CSV file saved in working directory |

### Key Notes:
- User Input Required: Steps 2 (file selection), 3 (parameters), 6.1 (strain verification)
- Output: Final CSV contains all analyzed tap response data

## 1. Importing Packages Required (No input required, just run)

In [ ]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import math
import os #<- package used to work with system file paths
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- another package used to plot graphs
from itertools import cycle #<- package used to iterate down rows (used in step 5 to add tap column)
import ipywidgets as widgets #<- widget tool to generate button and tab for graphs
from IPython.display import display #<- displays widgets
from ipyfilechooser import FileChooser
# from tkinter import Tk, filedialog #<- Tkinter is a GUI package
print("done step 1")

done step 1


## 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

**Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.**

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

**An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)**

In [2]:
starting_directory = '/Volumes'
chooser = FileChooser(starting_directory)
display(chooser)

FileChooser(path='/Volumes', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [3]:
print(chooser.selected_path)
folder_path=chooser.selected_path

/Users/gurmehak/Documents/RankinLab/Test_Datasets/PDScreen_TapHab_August15_2022/N2


In [4]:
screens = ['PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'Neuron_Genes_Screen']

screen_chooser = widgets.Select(options=screens, value=screens[0], description='Screen:')
display(screen_chooser)

Select(description='Screen:', options=('PD_Screen', 'ASD_Screen', 'G-Proteins_Screen', 'Glia_Genes_Screen', 'N…

In [ ]:
Screen=screen_chooser.value

## 3. User-Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Number of taps is pretty self-explanatory. How many taps does your experiment have? put in that number + 1 (N+1)!


3.2: Change your ISI number. This will be reflected in the name/title of the output figure.


**Note:** if you have different ISIs in the same folder, then come back and change this when you are graphing for the second set of data with the other ISI (Generally data from same ISIs are graphed together). If changing ISI mid-analysis, you can just skip straight to step 8 after running this cell again

In [ ]:
# 3.1 Input
number_of_taps = 30 # Taps in your experiment (N)

# 3.2 Input
ISI = 10  # ISI in your experiment
first_tap = 600 # when is your first tap? check your TRV files

In [22]:
# Here, open up one of the trv files to determine the times for each of these taps. 

# Record number of taps (N+1), e.g., if number_of_taps = 30, taps = [1, 2, 3, ..., 31]
taps = np.arange(1, number_of_taps+2).tolist()

# Assign tolerance to each tap
lower = np.arange(first_tap-2, first_tap-2+(number_of_taps*ISI), ISI) # (first tap, last tap+10s, ISI)
upper = np.arange(first_tap+2, first_tap+2+(number_of_taps*ISI), ISI) # (first tap, last tap+10s, ISI)
tolerances = [(int(l), int(u)) for l, u in zip(lower, upper)]
tolerances.append((1188,1191)) # (N+1)th tap

# Display taps with tolerances 
for i in taps:
    print(f"Tap {i}, tolerance: {tolerances[i-1]}")

print("done step 3")

Tap 1, tolerance: (598, 602)
Tap 2, tolerance: (658, 662)
Tap 3, tolerance: (718, 722)
Tap 4, tolerance: (778, 782)
Tap 5, tolerance: (838, 842)
Tap 6, tolerance: (898, 902)
Tap 7, tolerance: (958, 962)
Tap 8, tolerance: (1018, 1022)
Tap 9, tolerance: (1078, 1082)
Tap 10, tolerance: (1138, 1142)
Tap 11, tolerance: (1198, 1202)
Tap 12, tolerance: (1258, 1262)
Tap 13, tolerance: (1318, 1322)
Tap 14, tolerance: (1378, 1382)
Tap 15, tolerance: (1438, 1442)
Tap 16, tolerance: (1498, 1502)
Tap 17, tolerance: (1558, 1562)
Tap 18, tolerance: (1618, 1622)
Tap 19, tolerance: (1678, 1682)
Tap 20, tolerance: (1738, 1742)
Tap 21, tolerance: (1798, 1802)
Tap 22, tolerance: (1858, 1862)
Tap 23, tolerance: (1918, 1922)
Tap 24, tolerance: (1978, 1982)
Tap 25, tolerance: (2038, 2042)
Tap 26, tolerance: (2098, 2102)
Tap 27, tolerance: (2158, 2162)
Tap 28, tolerance: (2218, 2222)
Tap 29, tolerance: (2278, 2282)
Tap 30, tolerance: (2338, 2342)
Tap 31, tolerance: (1188, 1191)
done step 3


# 4. Constructing Filelist From Source File/Select File (Just run)

In [ ]:
#folder_path = '/Users/Joseph/Desktop/AVR14_10sISI' #- manual folder path if Tkinter is acting up

os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.trv'): # filters files with .trv (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list

if not filelist:
    raise FileNotFoundError("No .trv files found in the selected folder!")
else:
    print(f"Number of .trv files to process: {len(filelist)}")
    # print(f"Example of first and last file saved: {filelist[0]}, {filelist[-1]}") 

print('done step 4')

Number of .trv files to process: 5
done step 4


# 5. Process Data Function (Just Run)

In [24]:
def ProcessData(strain): 
    """
    Filters and processes .trv files matching the given strain.

    Parameters: 
        strain (str): keyword to match in the files

    Returns:
        dict: N (Plate number) and Dataframe with required columns ("time", "dura", "dist", "prob", "speed", "plate", "Date","Plate_id","Screen")

    """
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) # Finds the number of plates per strain
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier as number of plates = 0'.format(strain))
    else:
        pass
        print(f'Strain" {strain}')
        print(f'Number of plate: {Strain_N}') 

        # visiting files in this strain
        strain_filelist = [file for file in filelist if strain in file]
        df_list=[]
        for file in strain_filelist:
            if file.split('/')[-1].startswith('._'):
                pass
            else:
                print(f"File: {file}")
                df= pd.read_csv(file, sep=' ', header = None, encoding_errors='ignore')
                df['Plate_id'] = file.split('/')[-1].split('_')[-1].split('.')[0]
                df['Date'] = file.split('/')[-2].split('_')[0]
                df['Screen'] = file.split('/')[-4]
                df_list.append(df)
        DF_Total = pd.concat(df_list, ignore_index = True)

    # for f in strain_filelist:
    #     DF_Total = pd.concat(pd.read_csv(f, sep=' ', skiprows = 4, header = None))
    #     DF_Total = pd.concat([pd.read_csv(f, sep=' ', header = None, encoding_errors='ignore') for f in strain_filelist],
    #                   ignore_index=True) #<- imports your data files
    #     DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
        
        # column names for trv files
        DF_Total = DF_Total.rename( #<- more cleaning
                    {0:'time',
                    2:'rev_before',
                    3:'no_rev',
                    4:'stim_rev',
                    7:'dist',
                    8:'dist_std',
                    9:'dist_stderr',
                    11:'dist_0th',
                    12:'dist_1st',
                    13:'dist_2nd',
                    14:'dist_3rd',
                    15:'dist_100th',
                    18:'dura',
                    19:'dura_std',
                    20:'dura_stderr',
                    22:'dura_0th',
                    23:'dura_1st',
                    24:'dura_2nd',
                    25:'dura_3rd',
                    26:'dura_100th'}, axis=1)
        
        # check function here for NaN Columns
        DF_Total['plate'] = 0

        # Calculate reversal probability 
        DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) 

        # Calculate speed
        DF_Total['speed'] = DF_Total['dist']/DF_Total['dura']

        DF_Total_rows = int(DF_Total.shape[0])
        print(f'This strain/treatment has {DF_Total_rows} total taps') # Outputs as the second number. Check if you are missing taps!

        DF_Final = DF_Total[["time", "dura", "dist", "prob", "speed", "plate", "Date","Plate_id","Screen"]].copy()

    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Final': DF_Final}




def assign_taps(DF, tolerances):
    """
    Assigns tap number to each row in the DataFrame based on time tolerances.

    Parameters:
        df (pd.DataFrame): The DataFrame to modify
        tolerances (list of tuples): Each tuple is (lower, upper) time range

    Returns:
        None
    """
    DF['taps'] = np.nan
    for taps, tolerance in enumerate(tolerances): #[(99, 101), (109,111), ...]
        tap_lower,tap_upper = tolerance
        TimesInTapRange = DF['time'].between(tap_lower,tap_upper, inclusive="both")
        DF.loc[TimesInTapRange,'taps'] = taps+1 #set the tap to i where times are between



    
def insert_plates(df):   
    """
    Inserts a plate column into a dataframe.
    
    Parameters:
        df (pd.DataFrame): any dataframe
    
    Returns: 
        pd.DataFrame: dataframe with a plate column
    """
    df['plate']=(df['taps'] ==1).cumsum()


            
print('done step 5')

done step 5


# 6.1 Process Data (PLEASE READ, Add input here)

Here is the hardest part - From your naming convention, you need to pick a unique identifier for each group.

This means that all of names of your files for that strain should have that in common but is not commone with across all other files! If you did a good job naming your files and following a good naming convention, this should be easy.

## Be careful and really look hard in your naming structure. Note you want an unique identifier in the entire file path for the same group of files. An easy mistake is to have the strain name in the overall folder name, in this case if you use your strain name as a keyword it would include all files in that folder!

For example, if all your N2 files have a certain pattern like "N2_5x4" in this following example:
'/Users/Joseph/Desktop/AVR14_10sISI_TapHab_0710_2019/N2/20190710_141740/N2_5x4_f94h20c_100s30x10s10s_C0710ab.trv'

Then you need to set that identifier for the strain keyword:
'Strain_1' = 'N2_5x4'

## Depending on how many strains you are running for comparison, you may need to add/delete some lines!

## You are not naming your data groups here, we have a step for that later!
## Here, you want to note down ALL the strains you have in the folder

If you have just 2 strains, add hashtags (#) in front of the lines you do not need.
If you need more strains, just add more lines, following the same format!

In [9]:
genotype=[]
for f in filelist:
    genotype.append(f.split('/')[-3])

genotypes=np.unique(genotype)

StrainNames=dict(enumerate(genotypes,1))

print(f"Number of genotypes in the experiment: {len(genotypes)}")

# Display the first 5 Strain names in the experiment
for k in list(StrainNames)[:5]:
    print(f"{k}: {StrainNames[k]}")

Number of genotypes in the experiment: 1
1: N2


In [ ]:
print(list(StrainNames.values()))
print(type(list(StrainNames.values())))
print(type(list(StrainNames.values())[0]))
print('done step 6.1')

# 6.2 Process Data (just run this cell)

In [ ]:
# with open('/Volumes/JOSEPH/PD_Screen/F53B2.5_ok226/20220524_141642/ZE1_10x2_f72h20C_600s31x10s10s_B0520ba.trv') as f:
#     print(f)

In [ ]:
# import threading

In [ ]:
DataLists = [0]  #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in list(StrainNames.values()): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
#         threading.Thread(target=DataLists.append(ProcessData(s)['Final'])).start()
        DataLists.append(ProcessData(s)['Final']) #<- appends all data into a list of dataframes


for df in DataLists[1:]: 
    assign_taps(df, tolerances)
for df in DataLists[1:]:    
    insert_plates(df)
       
print('done step 6.2')

In [ ]:
# DataLists.append(ProcessData('vps-35_ok1880')['Final'])

In [ ]:
print(DataLists[1])


In [ ]:
print(len(genotypes))
print(len(np.unique(genotypes)))
print(len(DataLists))

# 7. Grouping Data and Naming (Optional: Add input here)

Here, you get to name your data groups/strain! Name your groups however you like under between the quotation marks for each strain.

For example: If your Strain1 is N2 and you wish for the group to be called N2,
your line should look like:

DataLists[x].assign(dataset = 'N2')

## Go back to step 6.1 to check which strain is which item on the DataLists.
In this example, the first item on DataLists is N2.


## Remember: Put your name in quotes. (ex: 'N2' and not N2)

As default, the names are set to the unique identifier labels.

## Depending on the number of strains you are running the comparison, you may have to delete/add lines of code (following the same format). 
## Remember to add/delete commas too.

# If you want to change your groups, you do that here. 
For example, if you have 5 strains in your folder but only want to compare between 2 or 3 strains, designate that here and follow through with steps 6 and 7. Once you are done, come back to step 6 and change your groups again (You are going to have to change your graph titles for the second run-through though)!

In [ ]:
TotalConcatenated=pd.DataFrame()
for d in range(1,len(np.unique(genotypes))+1):
    TotalConcatenated=pd.concat([TotalConcatenated,
                                 DataLists[d].assign(dataset=StrainNames.get(d))])

TotalConcatenated.reset_index(inplace=False)
print(TotalConcatenated)

#if TotalConcatenated["taps"].loc[ind] is not 1:
#   TotalConcatenated["taps"].loc[ind:indices[c+1]] = list(range(1,len(TotalConcatenated["taps"].loc[ind:indices[c+1]])+1))
# missing_taps(TotalConcatenated, accurate_taps, tolerances)

print('done step 7')

# Setting Colour Palette - Only run the below cell ONCE

The following code sets the colour palette for the whole experiment - and then designate one colour to each strain. After this, if as you are graphing you take away some strains, you can do so with the colours still matching accordingly.

In [ ]:
# print(TotalConcatenated['dataset'].str.split("_", n=1, expand=True))
TotalConcatenated['Screen']=Screen
print(TotalConcatenated)

In [ ]:
TotalConcatenated[['Gene', 'Allele']] = TotalConcatenated['dataset'].str.split('_', n=1, expand=True)
print(TotalConcatenated)

In [ ]:
TotalConcatenated['Allele']=TotalConcatenated['Allele'].fillna('N2')

In [ ]:
TotalConcatenated=TotalConcatenated.dropna()

In [ ]:
TotalConcatenated.to_csv(f'{TotalConcatenated.Screen[0].values[0]}_tap_output.csv')
print('done')


In [ ]:
# A debugging cell to test for strain 'XJ1' (which is the old N2)
print(TotalConcatenated[TotalConcatenated['Allele']=='XJ1'])

# Done!